# **AAHPS: ASSIGNMENT 5**

**Authors**: Nina Mislej and Nika Molan
<br/>**Student numbers**: 63200016 and 63200017

### **Exercise Description**
The aim of this assignment is to find the best result for **24 optimization** (*minimization*) **functions** that are available as **BBOB** (*Black-Box Optimization Benchmark*). We will implement **2 optimization programs** one of which has to be **local search** and the other one can be **any** optimization approach.
<br/>The functions are available in ***smoof*** package in **R** and we will initialize the functions with **40 dimensions** using the *iid*: **2023**

In addition to this report the results include the **coordinates** for each of the **24 minimums**. These are included in a separate file, one for each algorithm. 
<br/>One line represents **40**-**touple** for one function

In [1]:
# setting up the package and enviroment
import numpy as np
from rpy2.robjects import numpy2ri
from rpy2.robjects.packages import importr

numpy2ri.activate() # automatic conversion from numpy to R arrays
smoof = importr("smoof") # importing R smoof package
GenSA = importr("GenSA") # importing R GenSA package 

In [21]:
# initializing the functions
functions = {}

for fun in range(1, 25):
    functions[fun] = (smoof.makeBBOBFunction(40, fun, 2023))